# Dataless Neural Network for MIS finding in Graphs

## 1. Generate a random dense graph using NetworkX
This will be the graph that we find the MIS of

In [8]:
import os
import networkx as nx
import pickle

graph_directory = "./graphs/satlib"

datasets = []

for filename in os.listdir(graph_directory):
    if filename.endswith(".gpickle"):
        print("3SAT Graph ", os.path.join(graph_directory, filename), "is being imported ...")
        with open(os.path.join(graph_directory, filename), 'rb') as f:
            G = pickle.load(f)
            datasets.append(
            {
                "name": filename[:-8],
                "graph": nx.relabel.convert_node_labels_to_integers(
                    G, first_label=0
                ),
            })

# datasets = [{"name": "GNM10_1", "graph": nx.relabel.convert_node_labels_to_integers(nx.gnm_random_graph(50, 100), first_label=0)}]
            

3SAT Graph  ./graphs/satlib/CBS_k3_n100_m403_b50_58.gpickle is being imported ...
3SAT Graph  ./graphs/satlib/CBS_k3_n100_m441_b90_771.gpickle is being imported ...
3SAT Graph  ./graphs/satlib/CBS_k3_n100_m418_b10_512.gpickle is being imported ...
3SAT Graph  ./graphs/satlib/CBS_k3_n100_m423_b10_718.gpickle is being imported ...
3SAT Graph  ./graphs/satlib/CBS_k3_n100_m403_b50_737.gpickle is being imported ...
3SAT Graph  ./graphs/satlib/CBS_k3_n100_m418_b90_982.gpickle is being imported ...
3SAT Graph  ./graphs/satlib/CBS_k3_n100_m423_b50_757.gpickle is being imported ...
3SAT Graph  ./graphs/satlib/CBS_k3_n100_m403_b30_404.gpickle is being imported ...
3SAT Graph  ./graphs/satlib/CBS_k3_n100_m429_b50_277.gpickle is being imported ...
3SAT Graph  ./graphs/satlib/CBS_k3_n100_m423_b70_612.gpickle is being imported ...
3SAT Graph  ./graphs/satlib/CBS_k3_n100_m441_b10_727.gpickle is being imported ...
3SAT Graph  ./graphs/satlib/CBS_k3_n100_m441_b30_300.gpickle is being imported ...
3SAT 

# Graph Reduction using linear programming

In [9]:
for dataset in datasets:

    N_number_of_nodes_org = len(dataset["graph"].nodes)
    M_number_of_edges_org = len(dataset["graph"].edges)

    ###### Question: When we use LP reduction, do we always end up with a reduced graph that consists of disjoint subgraphs?
    ## Answer: not always

    components_before = [dataset["graph"].subgraph(c).copy() for c in nx.connected_components(dataset["graph"])]

    import numpy as np

    ################################# remove self loops
    self_loop_removal_cntr = 0
    for pair in list(dataset["graph"].edges):
        if pair[0] == pair[1]:
            dataset["graph"].remove_edge(pair[0],pair[1])
            self_loop_removal_cntr = self_loop_removal_cntr+1
            # print("self loop in node {} is removed".format([pair[0]]))
    # print("ALREADY REMOVED {} EDGES BECAUSE OF SELF LOOP".format(self_loop_removal_cntr))

    import cplex

    #############################################################
    ################### LP ###################
    #############################################################
    ########## INPUT: G
    ########## set of x_n = 1

    problem = cplex.Cplex()

    list_of_nodes = list(dataset["graph"].nodes)
    list_of_pairs_of_edges = list(dataset["graph"].edges)

    ### dictionary of id's
    node_id = {(n): 'node_id({0})'.format(n) for (n) in list_of_nodes}
    # print(node_id)
    problem.objective.set_sense(problem.objective.sense.maximize)
    problem.variables.add(names=list(node_id.values()),lb=[0.0]*len(node_id), ub=[1.0]*len(node_id))

    ## objective:
    problem.objective.set_linear(list(zip(list(node_id.values()), [1.0] * len(node_id))))

    ## constraint: for all (u,v)\in E, node_id(u) + node_id(v) <= 1
    """ Constraint (1) """
    for (u,v) in list_of_pairs_of_edges:
        #if u != v:
        lin_expr_vars_1 = []
        lin_expr_vals_1 = []
        lin_expr_vars_2 = []
        lin_expr_vals_2 = []
        lin_expr_vars_1.append(node_id[(u)])
        lin_expr_vals_1.append(1.0)
        lin_expr_vars_2.append(node_id[(v)])
        lin_expr_vals_2.append(1.0)
        problem.linear_constraints.add(lin_expr=[
            cplex.SparsePair(lin_expr_vars_1 + lin_expr_vars_2, val=lin_expr_vals_2 + lin_expr_vals_2)],
            rhs=[1.0], senses=["L"],
            names=['(1)_'])

    problem.set_log_stream(None)
    problem.set_results_stream(None)

    ## write program for testing
    # problem.write("LP_test.lp")
    problem.solve()

    if problem.solution.get_solution_type() == 0:
        print("CPLEX is outputting no solution exists")
    if problem.solution.get_solution_type() != 0:

        node_id_star = problem.solution.get_values()
        # print(node_id_star)
        ### removing nodes in (node_id_star == 1) along with their nieghbors
        nodes_to_be_removed = np.where(np.array(node_id_star)==1)

        # if len(nodes_to_be_removed[0])==0:
            # print("############# LP is solved , BUT without any nodes to be removed ##############")

        nodes_removed_from_LP_reduction=[]
        print(f"{dataset['name']} solution size: {len(nodes_to_be_removed[0])}")
        for node in nodes_to_be_removed[0]:
            # if node is still in graph
            if node in dataset["graph"].nodes:
                # find nieghbors
                nieghbor_list = list(dataset["graph"][int(node)])
                # remove node
                dataset["graph"].remove_node(node)
                nodes_removed_from_LP_reduction.append(node)
                # print("LP removing ", node)
                for nie_node in nieghbor_list:
                    dataset["graph"].remove_node(nie_node)
                    nodes_removed_from_LP_reduction.append(nie_node)
                    # print("LP removing ", nie_node, "since its a nieghbor of", node)

        # if len(dataset["graph"].nodes) == 0:
        #     print("THIS IS a CASE where a MIS is found with the LP with cardinality = ", np.count_nonzero(node_id_star))


        # re-label reduced graph from 0 to N
        dataset["graph"] = nx.relabel.convert_node_labels_to_integers(dataset["graph"])

        ### number to be added to the final MIS of the reduced graph is:
        # print("NUMBER of nodes in the MIS that we got from the LP", len(nodes_to_be_removed[0]))

    #######################################################################################################################
    #######################################################################################################################
    #######################################################################################################################

    N_number_of_nodes = len((dataset["graph"].nodes))
    M_number_of_edges = len((dataset["graph"].edges))

    M_number_of_edges_comp = (N_number_of_nodes*(N_number_of_nodes-1)/2) - M_number_of_edges

    print("Before LP {} - After LP  {}".format([N_number_of_nodes_org, M_number_of_edges_org],[N_number_of_nodes, M_number_of_edges]))

    # components_after = [dataset["graph"].subgraph(c).copy() for c in nx.connected_components(dataset["graph"])]

    # print(f"number of dis-joint subgraphs of G = [Before LP: {len(components_before)}, After LP: {len(components_after)}]")

CBS_k3_n100_m403_b50_58 solution size: 0
Before LP [1209, 4928] - After LP  [1209, 4928]
CBS_k3_n100_m441_b90_771 solution size: 0
Before LP [1323, 5586] - After LP  [1323, 5586]
CBS_k3_n100_m418_b10_512 solution size: 0
Before LP [1254, 5141] - After LP  [1254, 5141]
CBS_k3_n100_m423_b10_718 solution size: 0
Before LP [1269, 5276] - After LP  [1269, 5276]
CBS_k3_n100_m403_b50_737 solution size: 0
Before LP [1209, 4951] - After LP  [1209, 4951]
CBS_k3_n100_m418_b90_982 solution size: 0
Before LP [1254, 5181] - After LP  [1254, 5181]
CBS_k3_n100_m423_b50_757 solution size: 0
Before LP [1269, 5319] - After LP  [1269, 5319]
CBS_k3_n100_m403_b30_404 solution size: 0
Before LP [1209, 4913] - After LP  [1209, 4913]
CBS_k3_n100_m429_b50_277 solution size: 0
Before LP [1287, 5447] - After LP  [1287, 5447]
CBS_k3_n100_m423_b70_612 solution size: 0
Before LP [1269, 5322] - After LP  [1269, 5322]
CBS_k3_n100_m441_b10_727 solution size: 0
Before LP [1323, 5549] - After LP  [1323, 5549]
CBS_k3_n100